Setup

In [1]:
from spatial_italy.data import (
    request_confini_amministrativi_comuni,
    request_POSAS_2023_it_Comuni,
)
import json
import numpy as np

2023-08-02 16:03:43.964 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


# Data

In [2]:
COMUNI = [
    "Milano", 
    "Roma", 
    "Napoli", 
    "Castro",  # included to test a duplicate municipality names
    "Vandoies",  # included to test bilingual names
    ]

# Fixture

## POSAS_2023_it_Comuni

In [5]:
# Subset only for 2 age bins
df = request_POSAS_2023_it_Comuni()
sample = df[(df["Età"].isin([16,74])) & (df.Comune.isin(COMUNI))].to_dict(orient="tight")
sample

{'index': [181066,
  181124,
  193612,
  193670,
  281026,
  281084,
  482170,
  482228,
  521440,
  521498,
  618646,
  618704],
 'columns': ['Codice comune',
  'Comune',
  'Età',
  'Totale maschi',
  'Totale femmine'],
 'data': [[15146, 'Milano', 16, 6430, 5957],
  [15146, 'Milano', 74, 5691, 7572],
  [16065, 'Castro', 16, 4, 5],
  [16065, 'Castro', 74, 11, 17],
  [21110, 'Vandoies', 16, 14, 13],
  [21110, 'Vandoies', 74, 9, 13],
  [58091, 'Roma', 16, 13884, 13081],
  [58091, 'Roma', 74, 12599, 16446],
  [63049, 'Napoli', 16, 5283, 4962],
  [63049, 'Napoli', 74, 4318, 5340],
  [75096, 'Castro', 16, 10, 9],
  [75096, 'Castro', 74, 28, 12]],
 'index_names': [None],
 'column_names': [None]}

## Confini amministrativi comuni

In [3]:
gdf = request_confini_amministrativi_comuni()
sample = gdf[gdf.COMUNE.isin(COMUNI)]
sample.loc[:, "geometry"] = sample.simplify(1e4)  # Serialize less points
sample = json.loads(sample.to_json())
sample

## Sample upload files

In [25]:
gdf = request_confini_amministrativi_comuni()
sample = gdf[gdf.COMUNE.isin(COMUNI)]
# Only key column
sample = sample[["PRO_COM"]]
# Key column with different name from Istat standard
sample = sample.rename(columns={"PRO_COM":"Codice comune"})

In [26]:
# Random numerical values with NaN
sample = sample.assign(Random_value_1=np.random.randint(1, 10, len(sample)))
sample = sample.assign(Random_value_2=np.random.randint(100, 1000, len(sample)))
sample.loc[np.random.choice(sample.index), "Random_value_1"] = np.nan
sample.loc[np.random.choice(sample.index), "Random_value_2"] = np.nan

In [27]:
sample.to_csv("../tests/fixtures/upload_table_sample.csv", index=False)
sample.to_excel("../tests/fixtures/upload_table_sample.xlsx", index=False)